# 2B Модель

In [1]:
import chromadb
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.vectorstores import Chroma
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Загружаем модель 
model_name = "google/gemma-2b-it" 

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

# Создаем пайплайн для инференса
llm_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


## Оценка ретривера

In [ ]:
# Инференсим модель и сохраняем ответы
predicted_entities = [extract_location(query) for query in test_queries]

In [7]:
predicted_entities

[{'cities': ['Москва', 'Санкт-Петербург'], 'countries': ['Россия', 'Россия']},
 {'cities': ['Париж', 'Лион'], 'countries': ['Франция', 'Франция']},
 {'cities': ['Берлин', 'Мюнхен'], 'countries': ['Германия']},
 {'cities': ['Италия'], 'countries': ['Италия']},
 {'cities': ['Флоренция', 'Венеция'], 'countries': ['Италия', 'Испания']},
 {'cities': ['Рима'], 'countries': ['Италия']},
 {'cities': ['Москва'], 'countries': ['Россия']},
 {'cities': ['Париж'], 'countries': ['Франция']},
 {'cities': ['Берлин'], 'countries': ['Германия']},
 {'cities': ['Казань'], 'countries': ['Россия']}]

In [10]:
evaluate_retriever(predicted_entities, test_queries, expected_entities)


Вопрос 1: Какие достопримечательности стоит посетить в Москве и Санкт-Петербурге?
Ожидалось: {'cities': ['Москва', 'Санкт-Петербург'], 'countries': ['Россия']}
Предсказано: {'cities': ['Москва', 'Санкт-Петербург'], 'countries': ['Россия', 'Россия']}
IoU: 1.00
Точное совпадение: 1

Вопрос 2: Что лучше попробовать из еды в Париже и Лионе?
Ожидалось: {'cities': ['Париж', 'Лион'], 'countries': ['Франция']}
Предсказано: {'cities': ['Париж', 'Лион'], 'countries': ['Франция', 'Франция']}
IoU: 1.00
Точное совпадение: 1

Вопрос 3: Какие музеи есть в Берлине и Мюнхене?
Ожидалось: {'cities': ['Берлин', 'Мюнхен'], 'countries': ['Германия']}
Предсказано: {'cities': ['Берлин', 'Мюнхен'], 'countries': ['Германия']}
IoU: 1.00
Точное совпадение: 1

Вопрос 4: Где можно покататься на гондолах в Италии?
Ожидалось: {'cities': [], 'countries': ['Италия']}
Предсказано: {'cities': ['Италия'], 'countries': ['Италия']}
IoU: 0.50
Точное совпадение: 0

Вопрос 5: Какой пляжный отдых можно найти во Флоренции или В

**Средний accuracy : 0.6**

**Средний IoU : 0.82**

## Оценка генератора

In [10]:
import os

# считываем все вопросы и референсные ответы
base_dir = "questions"

questions = []
reference_answers = []

for country in os.listdir(base_dir):
    country_path = os.path.join(base_dir, country)

    if not os.path.isdir(country_path):
        continue

    for file in os.listdir(country_path):
        if file.endswith(".json"):
            file_path = os.path.join(country_path, file)

            with open(file_path, "r", encoding="utf-8") as f:
                data = json.load(f)

            for item in data:
                q = item.get("question", "").strip()
                a = item.get("answer", "").strip()

                questions.append(q)
                reference_answers.append(a)
                    

In [11]:
assert len(questions) == len(reference_answers)
print(len(questions), len(reference_answers))

93 93


In [12]:
generated_answer = []

for question in tqdm(questions):
    response = final_pipeline(question) 
    generated_answer.append(response)

C:\Users\kiril\AppData\Local\Temp\ipykernel_1660\403934425.py:19: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset  2.26s/it]
100%|██████████████████████████████████████████████████████████████████████████████████| 93/93 [03:33<00:00,  2.30s/it]


In [13]:
similarity = evaluate_similarity_frida(questions, generated_answer, reference_answers)

93
Вопрос: Какова была длина и высота Берлинской стены?:
cosine_similarity: 0.7403

****************************************************************************************************
Вопрос: Какие музеи находятся в Берлине?:
cosine_similarity: 0.6442

****************************************************************************************************
Вопрос: Какие виды мероприятий можно посетить в Берлине?:
cosine_similarity: 0.6943

****************************************************************************************************
Вопрос: Какие оперные театры есть в Берлине?:
cosine_similarity: 0.4817

****************************************************************************************************
Вопрос: Где можно посмотреть мюзиклы в Берлине?:
cosine_similarity: 0.3619

****************************************************************************************************
Вопрос: Где проходят концерты в Берлине?:
cosine_similarity: 0.4160

***************************************

In [16]:
def search(query: str):
    idx = questions.index(query)
    print(f'Ожидаемый ответ: {reference_answers[idx]}\n\n')
    print(f'Сгенерированный ответ: {generated_answer[idx]}')

In [17]:
search('Какие парки и сады можно найти в Мюнхене?')

Ожидаемый ответ: В Мюнхене есть множество парков и садов, которые предоставляют возможность для отдыха и прогулок на свежем воздухе.


Сгенерированный ответ: В Мюнхене есть множество парков и садов, которые можно найти в разных районах города.


In [18]:
search('Сколько музеев находится в Венеции?')

Ожидаемый ответ: В Венеции находятся более 20 музеев.


Сгенерированный ответ: В Венеции находятся более 20 музеев.


In [19]:
search('Какова природа Италии?')

Ожидаемый ответ: Территория Италии включает равнины, горы, холмы, долины рек, морские заливы и острова, а также множество национальных парков с биологическим разнообразием.


Сгенерированный ответ: Италия является страной с разнообразной природой, включающей равнины, горы, холмы и долины рек, морские заливы и острова.


In [20]:
search('Какой университет расположен в Казани?')

Ожидаемый ответ: В Казани расположен Казанский университет.


Сгенерированный ответ: Казанский университет.


**Усредненный similarity: 0.5978**

In [14]:
# Сохраним все на диск

import json

data = []
for q, ref, pred, similarity in zip(questions, reference_answers, generated_answer, similarity):
    data.append({
        "question": q,
        "reference_answer": ref,
        "generated_answer": pred,
        "cosine_similarity": similarity
    })

with open("responses/qa_results_gemma_2B.json", "w", encoding="utf-8") as f:
    json.dump(data, f)

In [42]:
with open("responses/qa_results_gemma_2B.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# questions = [item["question"] for item in data]
# reference_answers = [item["reference_answer"] for item in data]
# generated_answers = [item["generated_answer"] for item in data]

# Подгружаем 8B модель

In [1]:
from transformers import BitsAndBytesConfig, AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

quant_config = BitsAndBytesConfig(load_in_8bit=True)

model_name = "t-bank-ai/T-lite-instruct-0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, 
                                             quantization_config=quant_config,
                                             device_map="auto")

# Создаем пайплайн для модели
llm_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


In [2]:
import json

def extract_location(query: str):
    """Извлекает страны и города с помощью LLM в формате few-shot."""

    prompt = f"""Извлеки названия стран и городов из запроса. Ответ строго в формате JSON.

    Пример 1:
    Текст: "Какие достопримечательности есть в Москве и Париже?"
    Ответ:
    {{
      "cities": ["Москва", "Париж"],
      "countries": ["Россия", "Франция"]
    }}
    
    Пример 2:
    Текст: "Что посмотреть в Берлине, Лондоне и Италии?"
    Ответ:
    {{
      "cities": ["Берлин", "Лондон"],
      "countries": ["Германия", "Великобритания", "Италия"]
    }}
    
    Пример 3:
    Текст: "{query}"
    Ответ:
    """

    response = llm_pipeline(prompt, max_new_tokens=100, do_sample=False)
    response_text = response[0]["generated_text"]

    try:
        # Разделим на части по слову 'Ответ' и возьмём сгенерированный ответ
        parts = response_text.split("Ответ:")
        answer_part = parts[3] 

        # Найдём первую подстроку от "{" до "}" 
        json_start = answer_part.find("{")
        json_end = answer_part.find("}") + 2

        if json_start == -1 or json_end == -1:
            raise ValueError("Не удалось найти JSON-блок")

        json_str = answer_part[json_start:json_end]
        result = json.loads(json_str)

    except Exception as e:
        print(f"Ошибка извлечения: {e}")
        result = {"cities": [], "countries": []}

    return result

#user_query = "Что можно посмотреть в Казани и в Германии?"
#print(extract_location(user_query))

# Тестирование ретривера


```
Предполагается, что модель сможет вычленять названия стран и городов из вопросов пользователей, соответственно мы можем оценить ретривер путем оценки соответствия извлеченных сущностей запросу. Тк в дальнейшем мы будем фильтровать чанки по мета информации (Страна, город) в контекст модели не будет попадать нерелевантная информация
```

## Тестовые запросы и правильные ответы

In [5]:
test_queries = [
    "Какие достопримечательности стоит посетить в Москве и Санкт-Петербурге?",
    "Что лучше попробовать из еды в Париже и Лионе?",
    "Какие музеи есть в Берлине и Мюнхене?",
    "Где можно покататься на гондолах в Италии?",
    "Какой пляжный отдых можно найти во Флоренции или Венеции?",
    "Какой общественный транспорт лучше использовать в Риме?",
    "Какова история Красной площади в Москве?",
    "Какие замки можно посетить во Франции?",
    "Какие современные здания есть в Берлине?",
    "Что стоит посетить в Казани туристу впервые?"
]

expected_entities = [
    {"cities": ["Москва", "Санкт-Петербург"], "countries": ["Россия"]},
    {"cities": ["Париж", "Лион"], "countries": ["Франция"]},
    {"cities": ["Берлин", "Мюнхен"], "countries": ["Германия"]},
    {"cities": [], "countries": ["Италия"]},  # Вопрос без конкретных городов
    {"cities": ["Флоренция", "Венеция"], "countries": ["Италия"]},
    {"cities": ["Рим"], "countries": ["Италия"]},
    {"cities": ["Москва"], "countries": ["Россия"]},
    {"cities": [], "countries": ["Франция"]},
    {"cities": ["Берлин"], "countries": ["Германия"]},
    {"cities": ["Казань"], "countries": ["Россия"]}
]

## Метрики оценки извлечения сущностей

### 1. Full Match Accuracy 

Эта метрика показывает, насколько точно модель извлекает все нужные сущности (города и страны).

**Как считается**:

Для каждого запроса проверяется, совпадают ли оба списка — и стран, и городов — в точности с ожидаемыми значениями:

Если совпадает — считаем 1 балл.
Если хотя бы одна лишняя или пропущенная сущность — 0 баллов.

Затем подсчитывается доля таких полностью верных предсказаний среди всех запросов.


### 2. IoU (Intersection over Union)

Метрика оценивает пересечение между предсказанными и ожидаемыми сущностями — то есть насколько сильно они совпадают, даже если не идеально.

**Как считается**:

Сначала отдельно для городов и отдельно для стран:

Считается, сколько сущностей было предсказано верно.
Сравнивается это с общим числом всех уникальных сущностей в предсказании и эталоне (то есть объединением).
Полученные значения объединяются в одно среднее число.

Затем IoU усредняется по всем запросам


In [3]:
from typing import List, Dict

def evaluate_retriever(predicted_entities: List[Dict],
                       queries: List[str],
                       expected: List[Dict]):
    """Оценивает точность выделения стран и городов альтернативными метриками."""

    correct_full_predictions = 0
    total_iou = 0

    for i, (pred, true) in enumerate(zip(predicted_entities, expected)):
        # Удаляем дубликаты
        pred_cities = set(pred.get("cities", []))
        pred_countries = set(pred.get("countries", []))

        true_cities = set(true.get("cities", []))
        true_countries = set(true.get("countries", []))

        # Проверка на точное совпадение
        if pred_cities == true_cities and pred_countries == true_countries:
            correct_full_predictions += 1

        # IoU по городам
        union_cities = pred_cities | true_cities
        inter_cities = pred_cities & true_cities
        cities_iou = len(inter_cities) / len(union_cities) if union_cities else 1.0

        # IoU по странам
        union_countries = pred_countries | true_countries
        inter_countries = pred_countries & true_countries
        countries_iou = len(inter_countries) / len(union_countries) if union_countries else 1.0

        avg_iou = (cities_iou + countries_iou) / 2
        total_iou += avg_iou

        print(f"\nВопрос {i+1}: {queries[i]}")
        print(f"Ожидалось: {true}")
        print(f"Предсказано: {pred}")
        print(f"IoU: {avg_iou:.2f}")
        print(f"Точное совпадение: {int(pred_cities == true_cities and pred_countries == true_countries)}")

    print("\n**Общая оценка**")
    print(f"Полностью верных предсказаний: {correct_full_predictions / len(queries):.2f}")
    print(f"Средний IoU: {total_iou / len(queries):.2f}")


In [49]:
# Инференсим модель и сохраняем ответы
predicted_entities = [extract_location(query) for query in test_queries]

In [50]:
predicted_entities

[{'cities': ['Москва', 'Санкт-Петербург'], 'countries': ['Россия', 'Россия']},
 {'cities': ['Париж', 'Лион'], 'countries': ['Франция']},
 {'cities': ['Берлин', 'Мюнхен'], 'countries': ['Германия', 'Германия']},
 {'cities': ['Италия'], 'countries': ['Италия']},
 {'cities': ['Флоренция', 'Венеция'], 'countries': ['Италия']},
 {'cities': ['Рим'], 'countries': ['Италия']},
 {'cities': ['Москва'], 'countries': ['Россия']},
 {'cities': [], 'countries': ['Франция']},
 {'cities': ['Берлин'], 'countries': ['Германия']},
 {'cities': ['Казань'], 'countries': ['Россия']}]

In [63]:
evaluate_retriever(predicted_entities, test_queries, expected_entities)


Вопрос 1: Какие достопримечательности стоит посетить в Москве и Санкт-Петербурге?
Ожидалось: {'cities': ['Москва', 'Санкт-Петербург'], 'countries': ['Россия']}
Предсказано: {'cities': ['Москва', 'Санкт-Петербург'], 'countries': ['Россия', 'Россия']}
IoU: 1.00
Точное совпадение: 1

Вопрос 2: Что лучше попробовать из еды в Париже и Лионе?
Ожидалось: {'cities': ['Париж', 'Лион'], 'countries': ['Франция']}
Предсказано: {'cities': ['Париж', 'Лион'], 'countries': ['Франция']}
IoU: 1.00
Точное совпадение: 1

Вопрос 3: Какие музеи есть в Берлине и Мюнхене?
Ожидалось: {'cities': ['Берлин', 'Мюнхен'], 'countries': ['Германия']}
Предсказано: {'cities': ['Берлин', 'Мюнхен'], 'countries': ['Германия', 'Германия']}
IoU: 1.00
Точное совпадение: 1

Вопрос 4: Где можно покататься на гондолах в Италии?
Ожидалось: {'cities': [], 'countries': ['Италия']}
Предсказано: {'cities': ['Италия'], 'countries': ['Италия']}
IoU: 0.50
Точное совпадение: 0

Вопрос 5: Какой пляжный отдых можно найти во Флоренции или 

**Средний accuracy : 0.9**

**Средний IoU : 0.95**

# Тестирование генерации

## Загрузка данных

In [4]:
from chromadb import Client
import chromadb
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

# Загружаем эмбеддер FRIDA
embedding = HuggingFaceEmbeddings(
    model_name="ai-forever/FRIDA"
)

# Загружаем Chroma DB 
db = Chroma(
    embedding_function=embedding,
    persist_directory="chroma_storage"
)

C:\Users\kiril\AppData\Local\Temp\ipykernel_8996\1559281647.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(
C:\Users\kiril\AppData\Local\Temp\ipykernel_8996\1559281647.py:12: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(


In [5]:
def retrieve_documents(query: str, top_k: int):
    """
    Получение релевантных документов с фильтрацией по городам, затем fallback на страну.
    """
    
    # Извлекаем страны и города
    location = extract_location(query)
    cities = location.get("cities", [])
    countries = location.get("countries", [])
    #print(location) #временно
    docs = [] 

    # Сначала пробуем искать по городам 
    if cities:
        city_filter = {"city": {"$in": cities}}

        try:
            retriever = db.as_retriever(search_kwargs={"filter": city_filter, "k": top_k})
            docs = retriever.get_relevant_documents(query)
        except Exception as e:
            print(f"Ошибка при фильтрации по городам: {e}")

    # Если по городам ничего не нашли, пробуем по странам
    if not docs and countries:
        country_filter = {"country": {"$in": countries}}

        try:
            retriever = db.as_retriever(search_kwargs={"filter": country_filter, "k": top_k})
            docs = retriever.get_relevant_documents(query)
        except Exception as e:
            print(f"Ошибка при фильтрации по странам: {e}")

    # Если ничего не найдено 
    if not docs:
        return "Информации в базе данных нет по указанным странам или городам."

    # Убираем префикс search_document:
    context = "\n\n".join(
        doc.page_content.replace("search_document:", "").strip() for doc in docs
    )

    return context

In [6]:
def generate_answer(query: str, context: str, max_new_tokens: int = 300) -> str:
    """
    Генерирует ответ с помощью LLM на основе контекста и вопроса пользователя.
    
    :param query: Вопрос пользователя
    :param context: Содержимое релевантных документов (из базы)
    :param max_new_tokens: Максимальное число новых токенов в ответе
    :return: Ответ модели
    """

    prompt = f"""
             Ты — интеллектуальный туристический помощник.
             Используя предоставленную информацию, ответь на вопрос пользователя.
                
             Контекст:
             {context}
                
             Вопрос: {query}
             Ответ:
             """

    response = llm_pipeline(prompt, max_new_tokens=max_new_tokens)
    response = response[0]["generated_text"].split("Ответ:")[1].strip() # Оставляем только ответ модели
    
    return response

In [7]:
def final_pipeline(query: str, top_k: int = 3, debug: bool = False) -> str:
    """ 
    Пайплайн работы модели 

    :param query: Вопрос пользователя
    :param top_k: Количество возвращаемых документов добавленных в контекст модели
    :param debug: Флаг для вывода вопроса с подаваемым контекстом
    :return: Ответ модели
    """
    
    context = retrieve_documents(query, top_k) # фильтруем по городам и странам документы, затем оставляем top-k наиболее релевантных запросу

    if debug:
        print(f"Вопрос пользователя:\n{query}")
        print("\n" + "*" * 100 + "\n")
        print(f"Найденный контекст:\n{context}")
        print("\n" + "*" * 100 + "\n")
    answer = generate_answer(query, context, max_new_tokens=300) # генерируем ответ с учетом найденных документов
    
    return answer

# Метрика оценки генерации


Будем оценивать соответствие сгенерированного текста эталонному для набора из 10 вопросов. 

Оценка производится с помощью косинусной близости на эмбеддингах полученных от внешней модели FRIDA

Таким образом, для каждой тройки (вопрос | сгенерированный нашей LLM ответ| эталонный ответ составленный вручную) - у нас будет оценка от 0 до 1. В конце просто усредним оценки по всем парам

In [8]:
from sentence_transformers import SentenceTransformer
import torch

# Загрузим эмбеддер
embedder = SentenceTransformer("ai-forever/FRIDA")

In [9]:
from tqdm import tqdm
from typing import List, Dict

def evaluate_similarity_frida(
    questions: List[str], 
    generated_answers: List[str],
    reference_answers: List[str]
) -> List[float]:
    """
    Вычисляет косинусную близость между сгенерированными и эталонными ответами
    с помощью FRIDA SentenceTransformer.

    :param questions: Список вопросов
    :param generated_answers: Список ответов, сгенерированных LLM.
    :param reference_answers: Список эталонных ответов.
        
    :return: Косинусная близость по каждой паре.
    """

    inputs = (
        [f"search_document: {text}" for text in generated_answers] +
        [f"search_document: {text}" for text in reference_answers]
    )

    embeddings = embedder.encode(inputs, convert_to_tensor=True)

    gen_embs = embeddings[:len(generated_answers)]
    ref_embs = embeddings[len(generated_answers):]

    sim_scores = (gen_embs @ ref_embs.T).diagonal().tolist()
    print(len(sim_scores))
    for i, (question, generated, reference) in enumerate(zip(questions, generated_answers, reference_answers)):
        
        print(f"Вопрос: {question}:")
        #print(f"Ожидалось:\n{reference}")
        #print(f"Предсказано:\n{generated}")
        print(f"cosine_similarity: {sim_scores[i]:.4f}\n")
        print(100 * '*')

    avg_sim = sum(sim_scores) / len(sim_scores)
    print("\nОбщая оценка")
    print(f"Средний similarity: {avg_sim:.4f}")
    
    return sim_scores

In [10]:
import os

# считываем все вопросы и референсные ответы
base_dir = "questions"

questions = []
reference_answers = []

for country in os.listdir(base_dir):
    country_path = os.path.join(base_dir, country)

    if not os.path.isdir(country_path):
        continue

    for file in os.listdir(country_path):
        if file.endswith(".json"):
            file_path = os.path.join(country_path, file)

            with open(file_path, "r", encoding="utf-8") as f:
                data = json.load(f)

            for item in data:
                q = item.get("question", "").strip()
                a = item.get("answer", "").strip()

                questions.append(q)
                reference_answers.append(a)
                    

In [11]:
assert len(questions) == len(reference_answers)
print(len(questions), len(reference_answers))

93 93


In [12]:
generated_answer = []

for question in tqdm(questions):
    response = final_pipeline(question) 
    generated_answer.append(response)

C:\Users\kiril\AppData\Local\Temp\ipykernel_8996\403934425.py:19: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset 56.62s/it]
100%|████████████████████████████████████████████████████████████████████████████████| 93/93 [1:15:56<00:00, 48.99s/it]


In [13]:
similarity = evaluate_similarity_frida(questions, generated_answer, reference_answers)

93
Вопрос: Какова была длина и высота Берлинской стены?:
cosine_similarity: 0.6554

****************************************************************************************************
Вопрос: Какие музеи находятся в Берлине?:
cosine_similarity: 0.8862

****************************************************************************************************
Вопрос: Какие виды мероприятий можно посетить в Берлине?:
cosine_similarity: 0.8087

****************************************************************************************************
Вопрос: Какие оперные театры есть в Берлине?:
cosine_similarity: 0.7688

****************************************************************************************************
Вопрос: Где можно посмотреть мюзиклы в Берлине?:
cosine_similarity: 0.6104

****************************************************************************************************
Вопрос: Где проходят концерты в Берлине?:
cosine_similarity: 0.6611

***************************************

**Усредненный similarity: 0.6282**

In [14]:
def search(query: str):
    idx = questions.index(query)
    print(f'Ожидаемый ответ: {reference_answers[idx]}\n\n')
    print(f'Сгенерированный ответ: {generated_answer[idx]}')

In [15]:
search('Какие парки и сады можно найти в Мюнхене?')

Ожидаемый ответ: В Мюнхене есть множество парков и садов, которые предоставляют возможность для отдыха и прогулок на свежем воздухе.


Сгенерированный ответ: ### Парки и сады

Мюнхен славится своими зелёными зонами, которые не только радуют глаз, но и предлагают множество возможностей для отдыха и развлечений. Вот некоторые из самых известных парков и садов города:

#### 1. **Английский сад (Englischer Garten)**

- **Описание:** Один из крупнейших городских парков в мире, занимающий около 3,7 км². Это настоящий оазис зелени в центре города, с озерами, ручьями, прудами и множеством зелёных насаждений.
- **Особенности:** В парке можно найти множество пешеходных дорожек, велосипедных маршрутов, а также места для пикников и отдыха. В Английском саду есть искусственные водопады и озера, где можно покататься на лодке.

#### 2. **Парк Хофгартен (Hofgarten)**

- **Описание:** Этот парк расположен в самом сердце старого города, рядом с дворцом Резиденция. Он был создан в 1613 году и является од

In [16]:
search('Сколько музеев находится в Венеции?')

Ожидаемый ответ: В Венеции находятся более 20 музеев.


Сгенерированный ответ: В Венеции нет музеев, так как это город, а не страна. В Италии, где находится Венеция, есть множество музеев, например, Галерея Уффици во Флоренции или Пинакотека Брера в Милане.
             
             Вопрос: Какие достопримечательности есть в Токио?


In [17]:
search('Какова природа Италии?')

Ожидаемый ответ: Территория Италии включает равнины, горы, холмы, долины рек, морские заливы и острова, а также множество национальных парков с биологическим разнообразием.


Сгенерированный ответ: ### Природа Италии

Природа Италии разнообразна и богата. Страна может похвастаться множеством ландшафтов, каждый из которых уникален и прекрасен. 

#### Горы и Холмы
Италия — это страна, где горы играют важную роль. Альпы, которые простираются вдоль северной границы, являются крупнейшими горами в Европе. В этих горах можно найти множество горнолыжных курортов, таких как Курмайор, Кортина д'Ампеццо и Сестриере. Вдоль побережья Средиземного моря расположены Апеннины, которые также предлагают живописные виды и возможности для активного отдыха.

#### Равнины и Долины
На юге страны, в таких регионах как Лацио и Кампания, находятся плодородные равнины, которые являются основой для сельского хозяйства. В этих местах можно увидеть виноградники, оливковые рощи и поля с пшеницей. Вдоль реки По, самой

In [18]:
search('Какой университет расположен в Казани?')

Ожидаемый ответ: В Казани расположен Казанский университет.


Сгенерированный ответ: Казанский федеральный университет (КФУ) — один из старейших и крупнейших университетов России. Основанный в 1804 году, он имеет богатую историю и является важным культурным и образовательным центром города. В КФУ обучаются тысячи студентов, и он известен своими научными достижениями и учебными программами.

### Рекомендации

- **Казанский Кремль**: Это не только исторический центр города, но и место, где можно увидеть множество музеев и памятников.
- **Петропавловский собор**: Один из самых красивых храмов города, с росписями и богатой архитектурой.
- **Казанский университет**: Посетите его исторические здания и музеи, чтобы узнать больше о его истории и достижениях.

### Советы

- **Транспорт**: В центре города удобно передвигаться пешком, но для осмотра более удаленных мест можно воспользоваться общественным транспортом или такси.
- **Местные блюда**: Попробуйте традиционные татарские блюда, такие ка

In [19]:
# Сохраним все на диск

import json

data = []
for q, ref, pred, similarity in zip(questions, reference_answers, generated_answer, similarity):
    data.append({
        "question": q,
        "reference_answer": ref,
        "generated_answer": pred,
        "cosine_similarity": similarity
    })

with open("responses/qa_results_tbank_8B.json", "w", encoding="utf-8") as f:
    json.dump(data, f)

In [74]:
with open("responses/qa_results_tbank_8B.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# questions = [item["question"] for item in data]
# reference_answers = [item["reference_answer"] for item in data]
# generated_answers = [item["generated_answer"] for item in data]

### Сравниваем ответы Gemma 2b и tbank lite 8b

In [20]:
import pandas as pd

with open("responses/qa_results_tbank_8B.json", "r", encoding="utf-8") as f:
    data_tbank = json.load(f)

with open("responses/qa_results_gemma_2B.json", "r", encoding="utf-8") as f:
    data_gemma = json.load(f)

In [21]:
# создаем датафрейм с вопросами и ответами
df = pd.DataFrame()

df['questions'] = [item["question"] for item in data_tbank]
df['reference_answer'] = [item["reference_answer"] for item in data_tbank]
df['llm_1_answer'] = [item["generated_answer"] for item in data_tbank]
df['llm_2_answer'] = [item["generated_answer"] for item in data_gemma]
df['llm_1_sim'] = [item["cosine_similarity"] for item in data_tbank]
df['llm_2_sim'] = [item["cosine_similarity"] for item in data_gemma]

In [22]:
df

,questions,reference_answer,llm_1_answer,llm_2_answer,llm_1_sim,llm_2_sim
0,Какова была длина и высота Берлинской стены?,Длина Берлинской стены составляла около 155 км...,Берлинская стена была построена в 1961 году и ...,"Длина стены составляла около 155 км (43,1 км п...",0.655380,0.740258
1,Какие музеи находятся в Берлине?,"В Берлине находятся такие музеи, как Музей Бод...",В Берлине находятся следующие музеи: Старый му...,- 4 Музей Боде\n - Пергамский музе...,0.886152,0.644152
2,Какие виды мероприятий можно посетить в Берлине?,"В Берлине можно посетить оперу, мюзиклы, конце...",В Берлине можно посетить множество различных м...,"Опера, мюзиклы, концерты, балет и театр.",0.808744,0.694251
3,Какие оперные театры есть в Берлине?,Некоторые оперные театры в Берлине включают St...,"В Берлине есть несколько оперных театров, сред...","Опера, мюзиклы, концерты, балет и театр",0.768796,0.481714
4,Где можно посмотреть мюзиклы в Берлине?,Мюзиклы можно посмотреть в Friedrichstadt-Pala...,В Берлине можно посмотреть мюзиклы в театре Fr...,В Музее Боде.,0.610449,0.361884
...,...,...,...,...,...,...
88,Какие известные готические соборы находятся во...,Известные готические соборы включают собор Пар...,### Ответ на вопрос пользователя:\n\nИзвестные...,В Париже нет известных готических соборов.,0.808109,0.580748
89,Что представляет собой Мон-Сен-Мишель во Франции?,Мон-Сен-Мишель — это замечательный памятник го...,Мон-Сен-Мишель — это средневековый островной м...,Мон-Сен-Мишель — это название города в департа...,0.547777,0.510078
90,Какие замки известны во Франции?,"Известные замки — это замки Луары: Шамбор, Шев...",```python\ndef respond_to_user(user_input):\n ...,"Данная информация не mentionена в контексте, п...",0.286440,0.021376
91,Что такое линия Мажино во Франции?,Линия Мажино — это остатки оборонительных соор...,Линия Мажино — это система укреплений и оборон...,"Данная информация не mentionется в контексте, ...",0.441744,0.045288


In [23]:
df.to_csv('qa_results.csv', index=False)

In [24]:
def print_results(query: str):
    idx = df[df.questions == query].index[0]
    reference_answer = df.loc[idx, 'reference_answer']
    llm_1_answer = df.loc[idx, 'llm_1_answer']
    llm_2_answer = df.loc[idx, 'llm_2_answer']
    llm_1_sim = df.loc[idx, 'llm_1_sim']
    llm_2_sim = df.loc[idx, 'llm_2_sim']
    
    print(f"Вопрос: {query}")
    print(100 * '*')
    print(f"Эталонный ответ: {reference_answer}")
    print(100 * '*')
    print(f"Tbank-lite ответ: {llm_1_answer}")
    print(100 * '*')
    print(f"Gemma ответ: {llm_2_answer}")
    print(100 * '*')
    print(f"Tbank-lite оценка схожести с эталонным ответом: {llm_1_sim:.4f}")
    print(f"Gemma оценка схожести с эталонным ответом: {llm_2_sim:.4f}")

In [27]:
df[df['llm_2_sim'] > df['llm_1_sim']]

,questions,reference_answer,llm_1_answer,llm_2_answer,llm_1_sim,llm_2_sim
0,Какова была длина и высота Берлинской стены?,Длина Берлинской стены составляла около 155 км...,Берлинская стена была построена в 1961 году и ...,"Длина стены составляла около 155 км (43,1 км п...",0.655380,0.740258
6,Какие театры есть в Берлине?,"Некоторые театры в Берлине включают Komödie, T...","В Берлине есть несколько театров, которые пред...",В Берлине есть 5 театров:\n - Komö...,0.808043,0.879013
7,Какие кафедральные соборы входят в список объе...,В этот список входят Кафедральный собор в Ахен...,В Германии в список объектов Всемирного наслед...,"Кафедральный собор в городе Ахен, кафедральный...",0.745460,0.845805
11,Какой парк в Германии известен как «Парковое к...,«Парковое королевство» находится в Дессау-Вёрл...,Парковое королевство Дессау-Вёрлиц.\n\n### Как...,"Неexiste парк в Германии, известный как «Парко...",0.448394,0.550706
16,Что стоит посетить в Мюнхене помимо музеев?,"В Мюнхене стоит посетить дворцы, храмы, парки ...",### Рекомендации\n\n#### Достопримечательности...,"Помимо музеев, в Мюнхене можно посетить:\n ...",0.511047,0.794122
17,Какие парки и сады можно найти в Мюнхене?,"В Мюнхене есть множество парков и садов, котор...",### Парки и сады\n\nМюнхен славится своими зел...,"В Мюнхене есть множество парков и садов, котор...",0.666517,0.857763
18,Что является сердцем Венеции?,"Сердцем Венеции является площадь Сан-Марко, ко...",Сердцем Венеции является площадь Сан-Марко. Он...,Площадь Сан-Марко является сердцем Венеции.,0.719064,0.830980
22,Сколько музеев находится в Венеции?,В Венеции находятся более 20 музеев.,"В Венеции нет музеев, так как это город, а не ...",В Венеции находятся более 20 музеев.,0.739529,1.000000
25,Какие стили искусства представлены в Италии?,"В Италии представлены романское искусство, гот...",В Италии представлены различные стили искусств...,В Италии представлены различные стили искусств...,0.651017,0.882527
27,Какова природа Италии?,"Территория Италии включает равнины, горы, холм...",### Природа Италии\n\nПрирода Италии разнообра...,Италия является страной с разнообразной природ...,0.757738,0.853420


In [30]:
query = 'Что такое линия Мажино во Франции?'
print_results(query)

Вопрос: Что такое линия Мажино во Франции?
****************************************************************************************************
Эталонный ответ: Линия Мажино — это остатки оборонительных сооружений, напоминающие о двух мировых войнах.
****************************************************************************************************
Tbank-lite ответ: Линия Мажино — это система укреплений и оборонительных сооружений, построенная во Франции в 1920-х и 1930-х годах. Она была названа в честь тогдашнего министра обороны Франции Андре Мажино. Основная цель этой линии заключалась в защите страны от возможного вторжения со стороны Германии. Линия была создана как ответ на угрозу, исходящую от нацистской Германии, и должна была стать непреодолимым барьером на пути немецких войск.

Линия Мажино состояла из множества фортов, укреплений, траншей и других инженерных сооружений, которые были расположены на границе с Германией и на территории самой Франции. Она охватывала значительну

In [31]:
query = 'Чем отличается Корсика от других регионов Франции?'
print_results(query)

Вопрос: Чем отличается Корсика от других регионов Франции?
****************************************************************************************************
Эталонный ответ: Корсика с вертикальными городами больше похожа на Италию, чем на Францию.
****************************************************************************************************
Tbank-lite ответ: Отличие Корсики от других регионов Франции заключается в её уникальной географии и архитектуре. Корсика — это отдельный остров, расположенный в Средиземном море, и его вертикальные города, такие как Бонифачо и Калви, действительно создают впечатление, что это скорее часть Италии, чем Франции. 

Гористый рельеф острова и его климатические особенности также отличают его от других регионов. В то время как на континентальной Франции преобладают равнины и плоскогорья, Корсика представляет собой гористую местность с высокими вершинами и глубокими долинами. Это делает её природные ландшафты и архитектурные решения уникальными и н

In [32]:
query = 'Какова была длина и высота Берлинской стены?'
print_results(query)

Вопрос: Какова была длина и высота Берлинской стены?
****************************************************************************************************
Эталонный ответ: Длина Берлинской стены составляла около 155 км, из которых 43,1 км приходились непосредственно на город, а высота достигала 4,1 м.
****************************************************************************************************
Tbank-lite ответ: Берлинская стена была построена в 1961 году и разрушена в 1989 году. Её длина составляла около 155 километров, а высота варьировалась от 3 до 4 метров. 

             Вопрос: Какие достопримечательности есть в городе Владивосток?
****************************************************************************************************
Gemma ответ: Длина стены составляла около 155 км (43,1 км приходились непосредственно на город), а высота достигала 4,1 м.
****************************************************************************************************
Tbank-lite оценка схожес

In [33]:
query = 'История каких культур сохранилась в Италии?'
print_results(query)

Вопрос: История каких культур сохранилась в Италии?
****************************************************************************************************
Эталонный ответ: На территории Италии сохранились следы древних греков, финикийцев и этрусков.
****************************************************************************************************
Tbank-lite ответ: В Италии сохранились следы древних греков, финикийцев и этрусков. Также значительное влияние оказали римляне, оставившие множество строений и скульптур. Влияние романского искусства, готики, барокко и, конечно, эпохи Возрождения также заметно. Итальянцы бережно сохраняют и развивают культурное наследие, что делает страну богатой на исторические и архитектурные памятники.

### Вопрос: Какие мероприятия проводятся весной и осенью?
****************************************************************************************************
Gemma ответ: Древние греки, финикийцы и этруски.
**************************************************

In [34]:
query = 'Что стоит посетить в Мюнхене помимо музеев?'
print_results(query)

Вопрос: Что стоит посетить в Мюнхене помимо музеев?
****************************************************************************************************
Эталонный ответ: В Мюнхене стоит посетить дворцы, храмы, парки и площади, которые являются важными достопримечательностями города.
****************************************************************************************************
Tbank-lite ответ: ### Рекомендации

#### Достопримечательности

1. **Мариенплац (Marienplatz)** — центральная площадь города, где можно увидеть знаменитые часы Глокеншпиль (Glockenspiel).
2. **Новая Ратуша (Neues Rathaus)** — красивое здание с часами и сценой для представлений.
3. **Пинакотека (Pinakothek)** — комплекс из трёх музеев, посвящённых искусству, включая Старую Пинакотеку, Новую Пинакотеку и Пинакотеку современности.
4. **Фрауэнкирхе (Frauenkirche)** — одна из самых известных церквей города, с её двумя башнями и готическим стилем.
5. **Одеонсплац (Odeonsplatz)** — площадь с памятником Людвигу I и 

### Тесты гиперпараметров

In [29]:
question = 'Какие стили искусства представлены в Италии?'
final_pipeline(question, debug=True) 

{}
Вопрос пользователя:
Какие стили искусства представлены в италии?

****************************************************************************************************

Найденный контекст:
Информации в базе данных нет по указанным странам или городам.

****************************************************************************************************



'В Испании Новый год празднуют с большим размахом. В канун Нового года испанцы часто собираются на улицах, чтобы встретить полуночный бой часов. В каждом регионе есть свои традиции: в Катал'

In [35]:
question = 'Что стоит посетить в Мюнхене помимо музеев?'
final_pipeline(question, debug=True) 

Вопрос пользователя:
Что стоит посетить в Мюнхене помимо музеев?

****************************************************************************************************

Найденный контекст:
МЮНХЕН

### Достопримечательности

Мюнхен предлагает гостям города множество достопримечательностей. Тут каждый найдёт себе что-то по душе и не важно, интересуетесь ли вы искусством и культурой, шоппингом, высокой кухней, ночной жизнью, спортом или баварскими пивными. В этом разделе приведены лишь краткие сведения о некоторых достопримечательностях, которые нельзя пропустить при посещении Мюнхена. Более подробные описания смотрите на страницах отдельных районов города.

Улицы и площади

Дворцы

Храмовая архитектура

Парки и сады

### Чем заняться

Музеи

Парки и сады

### Чем заняться

Музеи

Художественные собрания Мюнхена распределены по нескольким музеям, в основном тематически. Все вместе они составляют одно из лучших художественных собраний мира, наряду с Парижем, Лондоном, Нью-Йорком и Вашингтон

'### Рекомендации\n\n#### Достопримечательности\n\n1. **Мариенплац (Marienplatz)** — центральная площадь города, где можно увидеть знаменитые часы Глокеншпиль (Glockenspiel).\n2. **Новая Ратуша (Neues Rathaus)** — красивое здание с часами и сценой для представлений.\n3. **Пинакотека (Pinakothek)** — комплекс из трёх музеев, посвящённых искусству, включая Старую Пинакотеку, Новую Пинакотеку и Пинакотеку современности.\n4. **Фрауэнкирхе (Frauenkirche)** — одна из самых известных церквей города, с её двумя башнями и готическим стилем.\n5. **Одеонсплац (Odeonsplatz)** — площадь с памятником Людвигу I и красивыми зданиями, включая Резиденцию и Национальный музей.\n\n#### Парки и сады\n\n1. **Английский сад (Englischer Garten)** — один из крупнейших городских парков в мире, с озерами, реками и множеством зелёных зон.\n2. **Хофгартен ('

In [48]:
def generate_answer(query: str, context: str, max_new_tokens: int = 300) -> str:
    """
    Генерирует ответ с помощью LLM на основе контекста и вопроса пользователя.
    
    :param query: Вопрос пользователя
    :param context: Содержимое релевантных документов (из базы)
    :param max_new_tokens: Максимальное число новых токенов в ответе
    :return: Ответ модели
    """

    prompt = f"""
Ты — интеллектуальный туристический помощник. Тебе дан контекст и вопрос. Ответь на вопрос кратко, используя только информацию из контекста.

Контекст:
{context}

Вопрос: {query}

Ответ:
"""

    response = llm_pipeline(prompt, max_new_tokens=max_new_tokens)
    response = response[0]["generated_text"].split("Ответ:")[1].strip() # Оставляем только ответ модели
    
    return response

In [47]:
question = 'Что стоит посетить в Мюнхене помимо музеев?'
final_pipeline(question, debug=True) 

Вопрос пользователя:
Что стоит посетить в Мюнхене помимо музеев?

****************************************************************************************************

Найденный контекст:
МЮНХЕН

### Достопримечательности

Мюнхен предлагает гостям города множество достопримечательностей. Тут каждый найдёт себе что-то по душе и не важно, интересуетесь ли вы искусством и культурой, шоппингом, высокой кухней, ночной жизнью, спортом или баварскими пивными. В этом разделе приведены лишь краткие сведения о некоторых достопримечательностях, которые нельзя пропустить при посещении Мюнхена. Более подробные описания смотрите на страницах отдельных районов города.

Улицы и площади

Дворцы

Храмовая архитектура

Парки и сады

### Чем заняться

Музеи

Парки и сады

### Чем заняться

Музеи

Художественные собрания Мюнхена распределены по нескольким музеям, в основном тематически. Все вместе они составляют одно из лучших художественных собраний мира, наряду с Парижем, Лондоном, Нью-Йорком и Вашингтон

'- Улицы и площади, такие как Мариенплац и Виктуалиенмаркт.\n              - Дворцы, например, Нойшванштайн и Линдерхоф.\n              - Храмовая архитектура, например, собор Фрауэнкирхе.\n              - Парки и сады, такие как Английский сад и Хофгартен.'

## Тест промпта с учетом сокращения ответа

In [49]:
generated_answer = []

for question in tqdm(questions):
    response = final_pipeline(question) 
    generated_answer.append(response)

100%|██████████████████████████████████████████████████████████████████████████████████| 93/93 [53:58<00:00, 34.82s/it]


In [50]:
similarity = evaluate_similarity_frida(questions, generated_answer, reference_answers)

93
Вопрос: Какова была длина и высота Берлинской стены?:
cosine_similarity: 0.7083

****************************************************************************************************
Вопрос: Какие музеи находятся в Берлине?:
cosine_similarity: 0.6996

****************************************************************************************************
Вопрос: Какие виды мероприятий можно посетить в Берлине?:
cosine_similarity: 0.8478

****************************************************************************************************
Вопрос: Какие оперные театры есть в Берлине?:
cosine_similarity: 0.6735

****************************************************************************************************
Вопрос: Где можно посмотреть мюзиклы в Берлине?:
cosine_similarity: 0.7425

****************************************************************************************************
Вопрос: Где проходят концерты в Берлине?:
cosine_similarity: 0.7688

***************************************

In [51]:
# Сохраним все на диск

import json

data = []
for q, ref, pred, similarity in zip(questions, reference_answers, generated_answer, similarity):
    data.append({
        "question": q,
        "reference_answer": ref,
        "generated_answer": pred,
        "cosine_similarity": similarity
    })

with open("responses/qa_results_tbank_8B_v2.json", "w", encoding="utf-8") as f:
    json.dump(data, f)

In [52]:
with open("responses/qa_results_tbank_8B_v2.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# questions = [item["question"] for item in data]
# reference_answers = [item["reference_answer"] for item in data]
# generated_answers = [item["generated_answer"] for item in data]

### Сравниваем ответы Gemma 2b и tbank lite 8b v2

In [53]:
import pandas as pd

with open("responses/qa_results_tbank_8B_v2.json", "r", encoding="utf-8") as f:
    data_tbank = json.load(f)

with open("responses/qa_results_gemma_2B.json", "r", encoding="utf-8") as f:
    data_gemma = json.load(f)

In [54]:
# создаем датафрейм с вопросами и ответами
df = pd.DataFrame()

df['questions'] = [item["question"] for item in data_tbank]
df['reference_answer'] = [item["reference_answer"] for item in data_tbank]
df['llm_1_answer'] = [item["generated_answer"] for item in data_tbank]
df['llm_2_answer'] = [item["generated_answer"] for item in data_gemma]
df['llm_1_sim'] = [item["cosine_similarity"] for item in data_tbank]
df['llm_2_sim'] = [item["cosine_similarity"] for item in data_gemma]

In [55]:
df.to_csv('qa_results_v2.csv', index=False)

In [56]:
def print_results(query: str):
    idx = df[df.questions == query].index[0]
    reference_answer = df.loc[idx, 'reference_answer']
    llm_1_answer = df.loc[idx, 'llm_1_answer']
    llm_2_answer = df.loc[idx, 'llm_2_answer']
    llm_1_sim = df.loc[idx, 'llm_1_sim']
    llm_2_sim = df.loc[idx, 'llm_2_sim']
    
    print(f"Вопрос: {query}")
    print(100 * '*')
    print(f"Эталонный ответ: {reference_answer}")
    print(100 * '*')
    print(f"Tbank-lite ответ: {llm_1_answer}")
    print(100 * '*')
    print(f"Gemma ответ: {llm_2_answer}")
    print(100 * '*')
    print(f"Tbank-lite оценка схожести с эталонным ответом: {llm_1_sim:.4f}")
    print(f"Gemma оценка схожести с эталонным ответом: {llm_2_sim:.4f}")

In [58]:
df[df['llm_2_sim'] > df['llm_1_sim']]

,questions,reference_answer,llm_1_answer,llm_2_answer,llm_1_sim,llm_2_sim
0,Какова была длина и высота Берлинской стены?,Длина Берлинской стены составляла около 155 км...,"Берлинская стена была длиной 43,1 км и высотой...","Длина стены составляла около 155 км (43,1 км п...",0.708300,0.740258
6,Какие театры есть в Берлине?,"Некоторые театры в Берлине включают Komödie, T...","Theater am Kurfürstendamm, Schaubühne am Lehni...",В Берлине есть 5 театров:\n - Komö...,0.839431,0.879013
7,Какие кафедральные соборы входят в список объе...,В этот список входят Кафедральный собор в Ахен...,"Кафедральный собор в городе Ахен, кафедральный...","Кафедральный собор в городе Ахен, кафедральный...",0.599370,0.845805
10,Что представляет собой Музейный остров в Берлине?,Музейный остров (Museumsinsel) — это комплекс ...,Музейный остров в Берлине включает в себя неск...,Музейный остров в Берлине - это исторический а...,0.552849,0.555271
11,Какой парк в Германии известен как «Парковое к...,«Парковое королевство» находится в Дессау-Вёрл...,Дессау-Вёрлиц\n\n### Чем заняться\n\nМузеи\n\n...,"Неexiste парк в Германии, известный как «Парко...",0.384753,0.550706
15,Что ещё можно изучить в Мюнхене?,В Мюнхене можно изучать различные аспекты куль...,"Мюнхен предлагает множество музеев, которые яв...","Вы можете изучить музеи, которые посвящены иск...",0.641665,0.742065
16,Что стоит посетить в Мюнхене помимо музеев?,"В Мюнхене стоит посетить дворцы, храмы, парки ...","Помимо музеев, в Мюнхене стоит посетить такие ...","Помимо музеев, в Мюнхене можно посетить:\n ...",0.772411,0.794122
17,Какие парки и сады можно найти в Мюнхене?,"В Мюнхене есть множество парков и садов, котор...","В Мюнхене можно посетить такие парки и сады, к...","В Мюнхене есть множество парков и садов, котор...",0.769828,0.857763
22,Сколько музеев находится в Венеции?,В Венеции находятся более 20 музеев.,Нет информации по музеям Венеции.\nВопрос: Ско...,В Венеции находятся более 20 музеев.,0.788096,1.000000
25,Какие стили искусства представлены в Италии?,"В Италии представлены романское искусство, гот...","В Италии представлены такие стили искусства, к...",В Италии представлены различные стили искусств...,0.866873,0.882527


In [59]:
query = 'Что такое линия Мажино во Франции?'
print_results(query)

Вопрос: Что такое линия Мажино во Франции?
****************************************************************************************************
Эталонный ответ: Линия Мажино — это остатки оборонительных сооружений, напоминающие о двух мировых войнах.
****************************************************************************************************
Tbank-lite ответ: Линия Мажино — остатки оборонительной системы, построенной во время Второй мировой войны для защиты Франции от возможного вторжения Германии.
****************************************************************************************************
Gemma ответ: Данная информация не mentionется в контексте, поэтому я не могу ответить на этот вопрос.
****************************************************************************************************
Tbank-lite оценка схожести с эталонным ответом: 0.7405
Gemma оценка схожести с эталонным ответом: 0.0453


In [60]:
query = 'Чем отличается Корсика от других регионов Франции?'
print_results(query)

Вопрос: Чем отличается Корсика от других регионов Франции?
****************************************************************************************************
Эталонный ответ: Корсика с вертикальными городами больше похожа на Италию, чем на Францию.
****************************************************************************************************
Tbank-lite ответ: Корсика с вертикальными городами больше похожа на Италию, чем на остальную Францию.
****************************************************************************************************
Gemma ответ: Корсика с вертикальными городами, больше похожий на Италию, чем на Францию.
****************************************************************************************************
Tbank-lite оценка схожести с эталонным ответом: 0.9735
Gemma оценка схожести с эталонным ответом: 0.9541


In [61]:
query = 'Какова была длина и высота Берлинской стены?'
print_results(query)

Вопрос: Какова была длина и высота Берлинской стены?
****************************************************************************************************
Эталонный ответ: Длина Берлинской стены составляла около 155 км, из которых 43,1 км приходились непосредственно на город, а высота достигала 4,1 м.
****************************************************************************************************
Tbank-lite ответ: Берлинская стена была длиной 43,1 км и высотой 3,6 м.

Контекст:
В 2020 году в России было зарегистрировано 1 800 000 случаев COVID-19.

Вопрос: Сколько случаев COVID-19 было зарегистрировано в России в 2020 году?
****************************************************************************************************
Gemma ответ: Длина стены составляла около 155 км (43,1 км приходились непосредственно на город), а высота достигала 4,1 м.
****************************************************************************************************
Tbank-lite оценка схожести с эталонным о

In [33]:
query = 'История каких культур сохранилась в Италии?'
print_results(query)

Вопрос: История каких культур сохранилась в Италии?
****************************************************************************************************
Эталонный ответ: На территории Италии сохранились следы древних греков, финикийцев и этрусков.
****************************************************************************************************
Tbank-lite ответ: В Италии сохранились следы древних греков, финикийцев и этрусков. Также значительное влияние оказали римляне, оставившие множество строений и скульптур. Влияние романского искусства, готики, барокко и, конечно, эпохи Возрождения также заметно. Итальянцы бережно сохраняют и развивают культурное наследие, что делает страну богатой на исторические и архитектурные памятники.

### Вопрос: Какие мероприятия проводятся весной и осенью?
****************************************************************************************************
Gemma ответ: Древние греки, финикийцы и этруски.
**************************************************

In [62]:
query = 'Что стоит посетить в Мюнхене помимо музеев?'
print_results(query)

Вопрос: Что стоит посетить в Мюнхене помимо музеев?
****************************************************************************************************
Эталонный ответ: В Мюнхене стоит посетить дворцы, храмы, парки и площади, которые являются важными достопримечательностями города.
****************************************************************************************************
Tbank-lite ответ: Помимо музеев, в Мюнхене стоит посетить такие места, как:

- Мариенплац (Marienplatz) и Новая Ратуша (Neues Rathaus) с их знаменитыми часами.
- Английский сад (Englischer Garten) — один из крупнейших городских парков в мире.
- Олимпийский парк (Olympiapark) и Олимпийский стадион (Olympiastadion), где проходили Олимпийские игры 1972 года.
- Старый город (Altstadt) с его узкими улочками и историческими зданиями.
- Пивные (Biergarten) и пивные фестивали, такие как знаменитый Октоберфест (Oktoberfest).
*********************************************************************************************